In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbs

In [14]:
test_path = 'datas/test.csv'
train_path = 'datas/train.csv'

test_pd = pd.read_csv(test_path)
train_pd = pd.read_csv(train_path)
train_pd.head(5)

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...


In [15]:
import csv

encoder_input_aug = []
f = open('datas/text2reconstruct.txt', 'r')
rdr = csv.reader(f)
for line in rdr:
    encoder_input_aug.append(line)
f.close

for text in encoder_input_aug[:3]:
    print(text)
    
len(encoder_input_aug)

['별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.']
['잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ']
['절테 간면 않 된는 굣 멥몫']


12952

In [16]:
train_input_set = []
for i, input in enumerate(train_pd['input']):
    train_input_set.append((train_pd['input'][i], train_pd['output'][i]))

len(train_input_set)

11263

In [17]:
encoder_input_aug[0]

['별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.']

# 형태소 치환함수

In [18]:
import random

# 초성, 중성, 종성 치환 쌍
similar_hangeul_pairs = {
    'ㄱ': 'ㅋ', 'ㅋ': 'ㄱ',
    'ㄴ': 'ㄹ', 'ㄹ': 'ㄴ',
    'ㅂ': 'ㅍ', 'ㅍ': 'ㅂ',
    'ㅏ': 'ㅓ', 'ㅓ': 'ㅏ',
    'ㅗ': 'ㅜ', 'ㅜ': 'ㅗ',
    'ㅆ': 'ㅉ', 'ㅉ': 'ㅆ'
}

# 초성, 중성, 종성 리스트
CHOSUNG_LIST = "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"
JUNGSUNG_LIST = "ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅛㅜㅠㅡㅣ"
JONGSUNG_LIST = [""] + list("ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ")

def decompose_hangeul(char):
    """한글 완성형 문자를 초성, 중성, 종성으로 분리"""
    if not ('가' <= char <= '힣'):
        return None, None, None

    base_code = ord(char) - 0xAC00
    initial = base_code // 588       # 초성
    medial = (base_code % 588) // 28 # 중성
    final = base_code % 28           # 종성

    return initial, medial, final

def compose_hangeul(initial, medial, final):
    """초성, 중성, 종성을 조합하여 한글 완성형 문자 생성"""
    return chr(0xAC00 + (initial * 588) + (medial * 28) + final)

def augment_hangeul_text(text, char_map, probability=0.3):
    result = []
    replace_candidates = []

    for i, char in enumerate(text):
        if '가' <= char <= '힣':
            initial, medial, final = decompose_hangeul(char)
            if initial is None or medial is None:  # 한글이 아닌 경우 예외 처리
                result.append(char)
                continue

            if random.random() < probability:
                # 초성, 중성, 종성 중 치환 가능한 부분 치환
                if initial < len(CHOSUNG_LIST) and CHOSUNG_LIST[initial] in char_map:
                    initial = CHOSUNG_LIST.index(char_map[CHOSUNG_LIST[initial]])
                    replace_candidates.append((i, initial, medial, final))
                elif medial < len(JUNGSUNG_LIST) and JUNGSUNG_LIST[medial] in char_map:
                    medial = JUNGSUNG_LIST.index(char_map[JUNGSUNG_LIST[medial]])
                    replace_candidates.append((i, initial, medial, final))
                elif final > 0 and final < len(JONGSUNG_LIST) and JONGSUNG_LIST[final] in char_map:
                    final = JONGSUNG_LIST.index(char_map[JONGSUNG_LIST[final]])
                    replace_candidates.append((i, initial, medial, final))

            result.append(compose_hangeul(initial, medial, final))

        else:
            result.append(char)

    # 치환이 한 번도 발생하지 않았을 경우 강제 치환
    if not replace_candidates:
        for i, char in enumerate(text):
            if '가' <= char <= '힣':
                initial, medial, final = decompose_hangeul(char)
                if initial is None or medial is None:  # 한글이 아닌 경우 예외 처리
                    continue

                if initial < len(CHOSUNG_LIST) and CHOSUNG_LIST[initial] in char_map:
                    initial = CHOSUNG_LIST.index(char_map[CHOSUNG_LIST[initial]])
                    result[i] = compose_hangeul(initial, medial, final)
                    break

    return ''.join(result)



In [19]:
# 테스트용 예시 텍스트 (문자열로 변경)
test_text = encoder_input_aug[0][0]
print("원본 텍스트:", test_text)

# 텍스트 치환 실행
augmented_text = augment_hangeul_text(test_text, similar_hangeul_pairs)
print("증강된 텍스트:", augmented_text)


원본 텍스트: 별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.
증강된 텍스트: 별 헌 게토 었껍떰. 왜 짜람듯닉 펼 1개늘 쥰눈징 컥깎폰 짜남믐놋섞 맒녹 섧망핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훌 굣. 깸삥읊 20여 년 댜력푠 곧 중 쩨윌 귑푼 낙밨떤 곶.


In [23]:
# 테스트용 예시 텍스트 (문자열로 변경)
test_text = train_input_set[1][1]
print("원본 텍스트:", test_text)

# 텍스트 치환 실행
augmented_text = augment_hangeul_text(test_text, similar_hangeul_pairs, probability=0.8)
print("증강된 텍스트:", augmented_text)


원본 텍스트: 잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
증강된 텍스트: 점먼 저코 칼 때 죃레요. 점왯돼 줌 ㅋ


## 데이터 증강 (인코더용, 디코더용 각각 다르게한다.)

In [21]:
# '''
# 디코더용 데이터증강

# 기존 데이터: 
# input, output

# 첫번째 증강 : 
# input -> input_aug

# 두번째 증강 : 
# output -> output_aug 

# 최종 augmentaion:
# input: output,
# input_aug : output,
# output_aug : output
# '''
# # train_input_set : (input, output) 쌍 리스트
# decoder_auged_set = []
# for raw_input, raw_output  in train_input_set:
#     # 첫번째 증강 : input -> input_aug
#     intput2aug = augment_hangeul_text(raw_input, similar_hangeul_pairs)
#     # 두번째 증강 : output -> output_aug 
#     output2aug = augment_hangeul_text(raw_output, similar_hangeul_pairs, probability=0.8)

#     decoder_auged_set.append((raw_input, raw_output))
#     decoder_auged_set.append((intput2aug, raw_output))
#     decoder_auged_set.append((output2aug, raw_output))

# # 데이터 증강된 텍스트 확인
# print(len(decoder_auged_set))
# for set in decoder_auged_set[:3]:
#     print(set[0], set[1])

# # 텍스트 판다스로 데이터프레임으로 만들기. ID,input,output colname형태로 저장하기.
# augmented_df = pd.DataFrame(decoder_auged_set, columns=["input", "output"])
# augmented_df.insert(0, "ID", ["AUG_{:05d}".format(i) for i in range(len(augmented_df))])
# # 데이터프레임 확인
# print(augmented_df.head())

# # 해당 데이터 프레임 csv파일로, 'datas/decoder_augmentation.csv' 로 저장하기
# output_file_path = 'datas/decoder_augmentation.csv'
# augmented_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')


In [24]:
encoder_input_aug

[['별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.'],
 ['잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ'],
 ['절테 간면 않 된는 굣 멥몫'],
 ['야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵겠댜! 한눈 쌀람한뗌많 쭈쳔. 탐패 냄쌕갊 묘둔 쟝졈울 까저갼눈 콧. 놂랙팡엣셔 칵좋 닮패왕 윳흥예 천렸욹 택 냐눈 넴쌘갸 꼐쏙 방웨 잊슴 ㅆ... 샨닉깎 할 맑 엽숨.'],
 ['집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 않추 탸툿했숲뉘닫. 휜닉쑵퍅끄왐 걸뤼툐 멂쥐 안았셔 좋앝쿄욥. 뱌룡 옆뻬 뼈뉘쩜믿 윗썰써 펀리헷숲닢댜. 따움엘토 읾옹할 쑤 있쓺먼 읾옹핥꼽 십넹욘. 예얌끼 여렵쮜만 ^^'],
 ['펀냔휜 잘 쉭곶 왔쑵닝따. 준윙에 맏쥡됴 만학썩 좋흖 겼 갇따용.'],
 ['쓰윔튿룸위 깝써칙갖 있네 ㅋ 꼭굽 췸규웨 낑펙투',
  ' 합뻔퓨 옥쪽',
  ' 캬뻬 깥은 띰 떼잎블',
  ' 녈븐 쇼팥예 커실',
  ' 부',
  ' 넹쟝코',
  ' 쫏쉭닿깝쥐 묘둔 궤 완벅캤닯.'],
 ['쨉빵뮨 의싸 쩐허 없쓺. 씬랏슬톄잎뾰댜 떠 짝음. 홈뗄 줄찾짱 15땜만 캉눙. 혹텔 1읾 쥬짜 옥굶 2많 원. 댜룬 윱로 쭈짰챵토 2만 언. 홍텔 냠찍언 뷸췬쩔.'],
 ['념묵 멎쥑교 꽁귀 좋습니닸. 췐곡윕뉘댜!'],
 ['짱졈: 쩡켤함',
  ' 츄챠 씨셜 죠흠',
  ' 쉼섦 좋음.   단젊: 줏짢 츌짜할 떼 뷸뻔함',
  ' 룸 섶피쓰 엾음',
  ' 줏편에 뭘갖 엎섞셔 샤려 갹귀 뷸펀함',
  ' 헤탕 충예 뭐까 잇눈쥐 엉언룟 셔 있얾 뷸편함',
  ' 좇식잎 섕갹뵤댜 벌롤읾.'],
 ['편힘 30퓬 넒계 쉰효 밭았써 긷타린는 쭐 봤욺면써 먈 얀항교 윗딱갸 거우 좌훽전을 받았섞 둘얹갼닙 쭈짰쟝 윕균셔 만짤략곡 팔료 똘럽본냅뉘따. 키타림먼 않돼냘

In [38]:
import os

'''
인코더용 데이터 증강

기존 데이터: 
input

데이터 증강: 
input -> input_aug1, 2

최종 augmentation:
input,
input_aug1,
input_aug2,
'''
# encoder_input_aug : train, test의 난독화 텍스트 합쳐놓은 문자열 리스트
encoder_auged_set = []
for raw_input  in encoder_input_aug:
    # 데이터 증강 input -> input_aug1, 2
    aug1 = augment_hangeul_text(raw_input[0], similar_hangeul_pairs)
    aug2 = augment_hangeul_text(raw_input[0], similar_hangeul_pairs)

    encoder_auged_set.append(str(raw_input[0]))
    encoder_auged_set.append(str(aug1))
    encoder_auged_set.append(str(aug2))



# 데이터 증강된 텍스트 확인
print(len(encoder_auged_set))
for augged in encoder_auged_set[:6]:
    print(augged)


# 텍스트 판다스로 데이터프레임으로 만들기. ID,input,output colname형태로 저장하기.
encoder_augmented_df = pd.DataFrame(encoder_auged_set, columns=["input"], )
# 데이터프레임 확인
print(encoder_augmented_df.head())

# 해당 데이터 프레임 csv파일로, 'datas/decoder_augmentation.csv' 로 저장하기
output_file_path = 'datas/encoder_augmentation.csv'
encoder_augmented_df.to_csv(output_file_path, index=True, encoding='utf-8-sig')

# # 'datas/encoder_augmentation.txt' 줄바꿈 문자로 구분해서 하나의 텍스트피일로 결과를 저장.
# os.makedirs('datas', exist_ok=True)
# output_file_path = 'datas/encoder_augmentation.txt'

# # 텍스트 파일로 저장
# with open(output_file_path, 'w', encoding='utf-8') as file:
#     for line in encoder_auged_set:
#         file.write(line + '\n')

# # 결과 확인용 출력
# print(f"{len(encoder_auged_set)}개의 데이터가 '{output_file_path}'에 저장되었습니다.")
# print("첫 5개 데이터:")
# print('\n'.join(encoder_auged_set[:5]))



38856
별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.
별 한 게토 었깝떰. 왜 싸람듯릭 펼 1개를 쥴눈징 컥깎폰 싸람믐놋섞 멂록 섧망헕쟈닐 탯끎뇬눈 녀뮤 퀼쿄... 야뭍툰 둠 편 덪씨 깍낀 싫훈 쿗. 깸삥읊 20여 년 댜녁푠 곧 중 쩨윌 귑분 낙밨떤 곶.
펼 한 케토 았깝떰. 왜 싸람듯닉 펼 1캐를 쥰룬징 컥꺾본 짜람믐롯섞 멂녹 섧멍헕쟈닐 탯끎룐룬 녀뮤 귈교... 야뭍툰 둠 변 닺찌 깍낄 싫훈 쿗. 깸삥읊 20여 년 댜녁뵨 콛 중 쩨윌 큅푼 낙팠떤 곶.
잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ
잚많 쟉꼬 칽 태 좋눼욯. 차뫳동 줆 ㅋ
잚많 쟉꽤 갉 태 좋눼욯. 차뫳동 줆 ㅋ
                                               input
0  별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...
1  별 한 게토 었깝떰. 왜 싸람듯릭 펼 1개를 쥴눈징 컥깎폰 싸람믐놋섞 멂록 섧망헕쟈...
2  펼 한 케토 았깝떰. 왜 싸람듯닉 펼 1캐를 쥰룬징 컥꺾본 짜람믐롯섞 멂녹 섧멍헕쟈...
3                             잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ
4                             잚많 쟉꼬 칽 태 좋눼욯. 차뫳동 줆 ㅋ


In [37]:
encoder_auged_set[:3]

['별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.',
 '별 한 게토 었깝떰. 왜 싸남듯릭 펼 1캐를 쥰눈징 컥꺾폰 짜남믐놋섞 맒녹 섧멍핥쟈닐 탯끎룐눈 녀뮤 귈교... 야뭍툰 둠 편 닺씨 깍낄 싫훌 굣. 깸삥읊 20여 년 댜녁푠 곧 중 쩨윌 귑분 낙밨떤 곶.',
 '별 헌 게토 았깝떰. 왜 짜람듯릭 별 1개를 쥰눈징 컥꺾폰 싸람믐롯섞 맒녹 삷망핥쟈닐 탯끎룐룬 녀뮤 퀼쿄... 야묵툰 둠 편 닺씨 꺽낄 싫훈 굣. 깸삥읊 20여 년 댜녁푠 콛 중 쎄윌 큅푼 낙팠떤 곶.']